# Modelagem preditiva
_Machine Learning_

---

## Sumário

1. **Importação de bibliotecas**
2. **Carregamento das bases**
3. **Análise dos dataframes**
4. **Modelagem preditiva**
    - 4.1. Preparação dos dados
    - 4.2. Treinamento dos modelos com todo o histórico de dados
    - 4.3. Treinamento dos modelos com histórico de dados a partir de 01-01-2014
    - 4.4. Treinamento dos modelos com histórico de dados a partir de 01-01-2015
    - 4.5. Comparativo dos resultados
5. **Tunagem dos hiperparâmetros dos melhores algoritmos**


<br>

---

<br>

## 1. Importação de bibliotecas

In [ ]:
# Importação de pacotes e definição de parâmetros globais

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import seaborn as sns
import warnings
import gc
import time
import optuna
import joblib
import lightgbm as lgb
import catboost as cb

from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from xgboost import XGBRegressor

from optuna.samplers import TPESampler
from optuna.study import MaxTrialsCallback
from optuna.trial import TrialState


In [ ]:
# Configurações para exibição de dados no Jupyter Notebook

# Configurar opção para exibir todas as linhas do Dataframe
pd.set_option('display.max_rows', None)

# Configurar para exibir o conteúdo completo das colunas
pd.set_option('display.max_colwidth', None)

# Configurar a supressão de mensagens de aviso durante a execução
warnings.filterwarnings('ignore')

# Configurar estilo dos gráficos do seaborn
sns.set_style('whitegrid')

## 2. Carregamento das bases

In [ ]:
# Efetuando a limpeza da memória antes do carregamento dos dados

print(f'\nQuantidade de objetos removidos da memória: {gc.collect()}')

In [ ]:
# Criando um dataframe a partir do arquivo train_001.csv

df_train = pd.read_csv('dados/train_001.csv', sep=',')
print('\nDATAFRAME: df_train')
df_train.head()

In [ ]:
# Criando um dataframe a partir do arquivo validation_001.csv

df_validation = pd.read_csv('dados/validation_001.csv', sep=',')
print('\nDATAFRAME: df_validation')
df_validation.head()

## 3. Análise dos dataframes

In [ ]:
# Exibindo a quantidade de linhas e colunas dos dataframes

# Criação de um dicionário com os dataframes e seus respectivos nomes
dfs = {
    'df_train': df_train,
    'df_validation': df_validation
}

# Iteração sobre o dicionário para exibir o nome e as dimensões dos dataframes
print(f'\nVOLUMETRIA')
for nome, df in dfs.items():
    print(f'\n{nome}')
    print(f'-'*45)
    print(f'Quantidade de linhas (registros):  {df.shape[0]}')
    print(f'Quantidade de colunas (variáveis): {df.shape[1]}')    

In [ ]:
# Função para geração de um dataframe de metadados

def gerar_metadados(dataframe):
    '''
    Gera um dataframe contendo metadados das colunas do dataframe fornecido.

    :param dataframe: Dataframe
        DataFrame para o qual os metadados serão gerados.
    :return: DataFrame
        DataFrame contendo os metadados.
    '''
    metadados = pd.DataFrame({
        'Variável': dataframe.columns,
        'Tipo': dataframe.dtypes,
        'Qtde de nulos': dataframe.isnull().sum(),
        '% de nulos': round((dataframe.isnull().sum()/len(dataframe))*100, 2),
        'Cardinalidade': dataframe.nunique(),
    })
    metadados = metadados.sort_values(by='Qtde de nulos', ascending=False)
    metadados = metadados.reset_index(drop=True)
    return metadados

In [ ]:
gerar_metadados(df_train)

## 4. Modelagem preditiva

### 4.1. Preparação dos dados

In [ ]:
# Separando as variáveis preditivas e a variável preditora (alvo)

features = df_train.columns.drop('Target')
target = 'Target'

In [ ]:
# Separando as variáveis numéricas e categóricas

numerical_features = df_train[features].select_dtypes(exclude=object).columns
categorical_features = df_train[features].select_dtypes(include=object).columns

In [ ]:
# Converter todas as colunas categóricas para string

df_train[categorical_features] = df_train[categorical_features].astype(str)
df_validation[categorical_features] = df_validation[categorical_features].astype(str)

In [ ]:
# Separando os dataframes com as variáveis preditivas e a variável preditora

X_train = df_train[features]
y_train = df_train[target]
X_test = df_validation[features]
y_test = df_validation[target]

In [ ]:
# Pré-processamento, transformação das features numéricas e categóricas

preprocessor = ColumnTransformer(
	transformers=[
		('num', StandardScaler(), numerical_features),
		('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
])

In [ ]:
# Verificando os shapes

print(f'Shape: X_train: {X_train.shape}, y_train: {y_train.shape}')
print(f'Shape: X_test:  {X_test.shape},  y_test: {y_test.shape}')

In [ ]:
# Verificando transformação de dados

X_train_transformed = preprocessor.fit_transform(X_train)
X_test_transformed = preprocessor.transform(X_test)

In [ ]:
# Verificando os shapes após transformação

print(f'Shape de X_train após transformação: {X_train_transformed.shape}')
print(f'Shape de X_test após transformação:  {X_test_transformed.shape}')

### 4.2. Treinamento dos modelos com todo o histórico de dados

In [ ]:
# Definindo os modelos a serem testados

models = {
    'LinearRegression': LinearRegression(),
    'LightGBM': lgb.LGBMRegressor(n_estimators=100, random_state=42, verbose=-1),
    'CatBoost': cb.CatBoostRegressor(iterations=100, depth=6, learning_rate=0.1, loss_function='RMSE', verbose=0),
    'GradientBoosting': GradientBoostingRegressor(n_estimators=100, random_state=42),
    'XGBoost': XGBRegressor(n_estimators=100, random_state=42, verbosity=0),
}

In [ ]:
def models_evaluation(models, X_train, y_train, X_test, y_test, preprocessor):
    '''
    Avalia modelos de aprendizado de máquina, calculando métricas de desempenho 
    no conjunto de treino e teste.

    :param models: dict
        Dicionário contendo os modelos a serem avaliados.
    :param X_train: DataFrame
        Conjunto de dados de treino com as variáveis independentes.
    :param y_train: Series
        Variável dependente para o conjunto de treino.
    :param X_test: DataFrame
        Conjunto de dados de teste com as variáveis independentes.
    :param y_test: Series
        Variável dependente para o conjunto de teste.
    :param preprocessor: ColumnTransformer
        Objeto de pré-processamento que será aplicado aos dados antes do treinamento do modelo.

    :return: list
        Lista contendo os resultados de avaliação de cada modelo, RMSE e MAE para treino 
        e teste, além do tempo de execução.
    '''
    results = []

    for model_name, model in models.items():
        # Cria um pipeline que combina o pré-processamento e o modelo
        pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', model)])

        # Registra o tempo de início
        start_time = time.time()
        
        # Treina o modelo usando o pipeline
        pipeline.fit(X_train, y_train)
        
        # Registra o tempo de término e calcula o tempo de execução
        end_time = time.time()
        elapsed_time = end_time - start_time

        # Previsões e cálculo das métricas no conjunto de treino
        train_predictions = pipeline.predict(X_train)
        train_rmse = np.sqrt(mean_squared_error(y_train, train_predictions))
        train_mae = mean_absolute_error(y_train, train_predictions)

        # Previsões e cálculo das métricas no conjunto de teste
        test_predictions = pipeline.predict(X_test)
        test_rmse = np.sqrt(mean_squared_error(y_test, test_predictions))
        test_mae = mean_absolute_error(y_test, test_predictions)

        # Adiciona os resultados à lista
        results.append((
            model_name, train_rmse, test_rmse, train_mae, test_mae, elapsed_time
        ))

    return results

In [ ]:
# Avaliando os modelos
results = models_evaluation(models, X_train, y_train, X_test, y_test, preprocessor)

# Criando DataFrame de resultados
results_df = pd.DataFrame(results, columns=[
                            'Modelo', 'RMSE em treino', 'RMSE em teste', 
                            'MAE em treino', 'MAE em teste', 'Tempo decorrido (s)'])

# Exibindo os resultados
results_df

### 4.3. Treinamento dos modelos com histórico de dados a partir de 01-01-2014

In [ ]:
# Selecionando dados a partir de 01-01-2014
df_train_00 = df_train[df_train['Date'] >= '2014-01-01'].copy()

# Converter todas as colunas categóricas para string
df_train_00[categorical_features] = df_train_00[categorical_features].astype(str)

# Separando os dataframes com as variáveis preditivas e a variável preditora
X_train = df_train_00[features]
y_train = df_train_00[target]

# Pré-processamento, transformação das features numéricas e categóricas
preprocessor = ColumnTransformer(
	transformers=[
		('num', StandardScaler(), numerical_features),
		('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
])

# Avaliando os modelos
results = models_evaluation(models, X_train, y_train, X_test, y_test, preprocessor)

# Criando DataFrame de resultados
results_df_00 = pd.DataFrame(results, columns=[
                            'Modelo', 'RMSE em treino', 'RMSE em teste', 
                            'MAE em treino', 'MAE em teste', 'Tempo decorrido (s)'])

# Exibindo os resultados
results_df_00

### 4.4. Treinamento dos modelos com histórico de dados a partir de 01-01-2015

In [ ]:
# Selecionando dados a partir de 01-01-2015
df_train_01 = df_train[df_train['Date'] >= '2015-01-01'].copy()

# Converter todas as colunas categóricas para string
df_train_01[categorical_features] = df_train_01[categorical_features].astype(str)

# Separando os dataframes com as variáveis preditivas e a variável preditora
X_train = df_train_01[features]
y_train = df_train_01[target]

# Pré-processamento, transformação das features numéricas e categóricas
preprocessor = ColumnTransformer(
	transformers=[
		('num', StandardScaler(), numerical_features),
		('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
])

# Avaliando os modelos
results = models_evaluation(models, X_train, y_train, X_test, y_test, preprocessor)

# Criando DataFrame de resultados
results_df_01 = pd.DataFrame(results, columns=[
                            'Modelo', 'RMSE em treino', 'RMSE em teste', 
                            'MAE em treino', 'MAE em teste', 'Tempo decorrido (s)'])

# Exibindo os resultados
results_df_01

### 4.5. Comparativo dos resultados

In [ ]:
# Utilizando todo o histórico de dados

# Exibindo os resultados
results_df

In [ ]:
# Utilizando histórico de dados a partir de 01-01-2014

# Exibindo os resultados
results_df_00

In [ ]:
# Utilizando histórico de dados a partir de 01-01-2015

# Exibindo os resultados
results_df_01

Os valores elevados de **RMSE** e **MAE** podem ser atribuídos à grande variação na variável dependente _'Target'_, que inclui dias com vendas zero e dias com vendas muito elevadas. Esse cenário reflete a realidade de que drogarias podem estar fechadas ou operar em horários reduzidos, o que influencia os dados. O foco, no entanto, deve ser nas diferenças entre as métricas de treino e teste, o que indica a capacidade dos modelos em generalizar.

Dentre os modelos analisados, o **LinearRegression** teve o pior desempenho com os maiores valores de erro. O **GradientBoosting** apresentou boas diferenças entre treino e teste, mas os erros ainda foram elevados. O **CatBoost** se destacou com métricas razoáveis, mas seu tempo de execução foi mais alto. O **XGBoost** obteve os menores valores de _RMSE_ e _MAE_, porém com uma diferença maior entre treino e teste, indicando possível overfitting. O **LightGBM**, com _RMSE_ e _MAE_ competitivos e o menor tempo de execução, mostrou-se o modelo mais equilibrado, oferecendo a melhor performance geral.

## 5. Tunagem dos hiperparâmetros dos melhores algoritmos

### 5.1. Tunagem do modelo LightGBM com Optuna

In [ ]:
def objective(trial):
    '''
    Função objetivo para otimizar os hiperparâmetros do modelo LGBMRegressor usando o Optuna.

    :param trial: optuna.Trial
        Objeto que sugere valores para os hiperparâmetros do modelo durante o processo de otimização.

    :return: float
        O valor do MAE calculado no conjunto de teste para o modelo treinado.
    '''
    model = lgb.LGBMRegressor(
        # Número de árvores no modelo, cada árvore corrige a anterior
        n_estimators=trial.suggest_int('n_estimators', 100, 1000),
        
        # Tamanho dos passos que o modelo dá ao ajustar os pesos para minimizar o erro
        learning_rate=trial.suggest_float('learning_rate', 1e-3, 0.1, log=True),
        
        # Número máximo de folhas (ou nós terminais) em cada árvore
        num_leaves= trial.suggest_int('num_leaves', 5, 500),
        
        # Limita a profundidade máxima de cada árvore
        max_depth=trial.suggest_int('max_depth', 3, 20),
        
        # Controla a fração de dados usados para treinar cada árvore
        subsample=trial.suggest_float('subsample', 0.5, 1),
        
        # Define a fração de colunas (features) usadas para construir cada árvore
        colsample_bytree=trial.suggest_float('colsample_bytree', 0.5, 1),
        
        # Define o número mínimo de amostras necessárias para formar uma folha
        min_data_in_leaf=trial.suggest_int('min_data_in_leaf', 1, 300),
        
        # Regularização L1 (Lasso)- penalidade proporcional ao valor absoluto dos coeficientes
        reg_alpha=trial.suggest_float('reg_alpha', 1e-8, 1, log=True),
        
        # Regularização L2 (Ridge) - penalidade proporcional ao quadrado dos coeficientes
        reg_lambda=trial.suggest_float('reg_lambda', 1e-8, 1, log=True),
        
        # Semente aleatória para garantir reprodutibilidade dos resultados     
        random_state=42,
        
        # Silenciar a saída de logs durante o treinamento
        verbose=-1,         
    )

    # Cria um pipeline que combina o pré-processamento e o modelo
    pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', model)])
    
    # Treina o modelo usando o pipeline
    pipeline.fit(X_train, y_train)
   
    # Previsões e cálculo das métricas no conjunto de teste
    test_predictions = pipeline.predict(X_test)
    test_mae = mean_absolute_error(y_test, test_predictions)
    
    return test_mae

In [ ]:
# Cria o estudo Optuna com o sampler TPE
study = optuna.create_study(direction='minimize', sampler=TPESampler(seed=42))
study.optimize(objective, n_trials=75)

In [ ]:
print(f'Melhores hiperparâmetros: {study.best_params}')
print(f'\nMelhor MAE: {study.best_value:.4f}')

In [ ]:
df_best_params_00 = pd.DataFrame([study.best_params])
df_best_params_00

### 5.2. Tunagem do modelo XGBoost com Optuna

In [ ]:
def objective(trial):
    '''
    Função objetivo para otimizar os hiperparâmetros do modelo XGBRegressor usando o Optuna.

    :param trial: optuna.Trial
        Objeto que sugere valores para os hiperparâmetros do modelo durante o processo de otimização.

    :return: float
        O valor do MAE calculado no conjunto de teste para o modelo treinado.
    '''
    model = XGBRegressor(
        # Número de árvores (boosting rounds)
        n_estimators=trial.suggest_int('n_estimators', 100, 1000),
        
        # Taxa de aprendizado
        learning_rate=trial.suggest_float('learning_rate', 1e-3, 0.1, log=True),
        
        # Número máximo de folhas por árvore
        max_leaves=trial.suggest_int('max_leaves', 5, 500),
        
        # Limita a profundidade máxima de cada árvore
        max_depth=trial.suggest_int('max_depth', 3, 20),
        
        # Proporção de amostras usadas para treinar cada árvore (controle de overfitting)
        subsample=trial.suggest_float('subsample', 0.5, 1),
        
        # Proporção de colunas (features) usadas para construir cada árvore
        colsample_bytree=trial.suggest_float('colsample_bytree', 0.5, 1),
        
        # Proporção de colunas usadas por cada nó para divisão (split)
        colsample_bylevel=trial.suggest_float('colsample_bylevel', 0.5, 1),
        
        # Regularização L1 (Lasso) - penalidade no valor absoluto dos coeficientes
        reg_alpha=trial.suggest_float('reg_alpha', 1e-8, 1, log=True),
        
        # Regularização L2 (Ridge) - penalidade no valor quadrático dos coeficientes
        reg_lambda=trial.suggest_float('reg_lambda', 1e-8, 1, log=True),
        
        # Peso mínimo necessário para formar uma folha
        min_child_weight=trial.suggest_int('min_child_weight', 1, 300),
        
        # Semente aleatória para garantir reprodutibilidade dos resultados
        random_state=42,
        
        # Silenciar a saída de logs durante o treinamento
        verbosity=0,
    )

    # Cria um pipeline que combina o pré-processamento e o modelo
    pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', model)])
    
    # Treina o modelo usando o pipeline
    pipeline.fit(X_train, y_train)
   
    # Faz previsões no conjunto de teste e calcula as métricas
    test_predictions = pipeline.predict(X_test)
    test_mae = mean_absolute_error(y_test, test_predictions)
    
    return test_mae

In [ ]:
# Cria o estudo Optuna com o sampler TPE
study = optuna.create_study(direction='minimize', sampler=TPESampler(seed=42))
study.optimize(objective, n_trials=75)

In [ ]:
print(f'Melhores hiperparâmetros: {study.best_params}')
print(f'\nMelhor MAE: {study.best_value:.4f}')

In [ ]:
df_best_params_01 = pd.DataFrame([study.best_params])
df_best_params_01